# MMSP Demo Script
This is a demo script for the paper -- "A Sharpness Based Loss Function for Removing Out-of-Focus Blur"

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import pandas as pd
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras
import numpy as np 
keras.utils.set_random_seed(0)
np.random.seed(0)
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Model
from tqdm.notebook import tqdm #.notebook 
import time 
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, Flatten, Dense, concatenate, Reshape, UpSampling2D, Activation
from tensorflow.keras.layers import Dropout, BatchNormalization, Input, GlobalAveragePooling2D, MaxPooling2D, Concatenate
from keras.models import load_model
from dataGeneration import *
from restorationNetwork import *
import cv2 
from correctedQ_TF import *
from tensorflow.image import psnr, ssim
from contextlib import redirect_stdout
import datetime
import PIL.Image
import concurrent.futures
import time
from keras.preprocessing.image import load_img, img_to_array, save_img
import os

## Directory Setup

In [ ]:
# Defining clean folder path
CLEAN_IMG_PATH = 'CleanImages'

# Defining path for storing blurry images
BLURRY_IMG_PATH = 'BlurryImages'

# Create folder if not created
if (os.path.exists(BLURRY_IMG_PATH) == False):
    os.mkdir(BLURRY_IMG_PATH)

## Generate Degraded Images

In [ ]:
# Function to generate blurry images
def blurImages(filename, kernel_size):
    CLEAN_IMG_PATH = 'CleanImages'
    BLURRY_IMG_PATH = 'BlurryImages'

    # Load image
    I_original = PIL.Image.open(CLEAN_IMG_PATH + '/' + filename)

    # Blur image
    I_blurry = cv2.blur(
        np.array(I_original), 
        (kernel_size, kernel_size)
        )
    
    I_degraded = PIL.Image.fromarray(I_blurry)
    I_degraded.save(BLURRY_IMG_PATH + '/' + filename)

# Handler for blurrying images
def blurImages_args(args):
    return blurImages(*args)

clean_images = os.listdir(CLEAN_IMG_PATH)
blur_params = [(file_name, 5) for file_name in clean_images]

with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(blurImages_args, blur_params)

# Restore image with Pure MAE Net

In [ ]:
# Set path for pre-trained model
pureMAE_model_PATH = 'preTrainedModels/pureMAE_model.h5'

# Set restored path
RESTORED_PATH = 'RestoredImages'
if(os.path.exists(RESTORED_PATH) == False):
    os.mkdir(RESTORED_PATH)

# Load restoration model
pureMAE_model = load_model(
    pureMAE_model_PATH,
    custom_objects={
        'PSNRMetric' : PSNRMetric,
        'SSIMMetric' : SSIMMetric
    }
)

for filename in tqdm(os.listdir(BLURRY_IMG_PATH), desc='Restoring Blurry Images'):
    # Load image
    I_blurry = load_img(os.path.join(BLURRY_IMG_PATH, filename), color_mode = 'grayscale')
    I_blurry_array = img_to_array(I_blurry)
    I_blurry_array = np.array([I_blurry_array]) / 255.0

    # Restore Image
    I_restored = pureMAE_model.predict(I_blurry_array)
    I_restored = np.clip(I_restored, 0, 1)

    # Normalize restored image
    I_restored_scaled = (I_restored[0] * 255).astype('uint8')

    # Create new extension for restored image
    new_filename = filename[:-4] + '_pureMAE_restored.png'
    save_img(
        os.path.join(RESTORED_PATH, new_filename),
        I_restored_scaled,
        scale = False
    )

## Restore image with Fine-Tuned Models

In [ ]:
# Set path for fine-tuned models (two versions)
fineTunedE9_model_PATH = 'preTrainedModels/fineTunedModel_E9.h5'
fineTunedE11_model_PATH = 'preTrainedModels/fineTunedModel_E11.h5'
MAE_loss = keras.losses.MeanAbsoluteError()

In [ ]:
def QMetric(y_true, y_pred):
    return -1 * compute_Q_TF(y_pred)

def wrapperLoss(y, X):
    pass
    # mae_loss_val = MAE_loss(X, y)
    # kappa =  0.01
    # Q_val = -1 * compute_Q_TF_withCD(X)
    # final_loss = mae_loss_val + kappa * Q_val
    # return final_loss

In [ ]:

# Set restored path
RESTORED_PATH = 'RestoredImages'
if(os.path.exists(RESTORED_PATH) == False):
    os.mkdir(RESTORED_PATH)

# Load restoration model (E9)
fineTunedE9_model = load_model(
    fineTunedE9_model_PATH,
    custom_objects={
        'PSNRMetric' : PSNRMetric,
        'SSIMMetric' : SSIMMetric,
        'QMetric' : QMetric,
        'wrapperLoss' : wrapperLoss
    }
)

# Load restoration model (E11)
fineTunedE11_model = load_model(
    fineTunedE11_model_PATH,
    custom_objects={
        'PSNRMetric' : PSNRMetric,
        'SSIMMetric' : SSIMMetric,
        'QMetric' : QMetric,
        'wrapperLoss' : wrapperLoss
    }
)


for filename in tqdm(os.listdir(BLURRY_IMG_PATH), desc='Restoring Blurry Images'):
    # Load image
    I_blurry = load_img(os.path.join(BLURRY_IMG_PATH, filename), color_mode = 'grayscale')
    I_blurry_array = img_to_array(I_blurry)
    I_blurry_array = np.array([I_blurry_array]) / 255.0

    # Restore Image (E9)
    I_restored_E9 = fineTunedE9_model.predict(I_blurry_array)
    I_restored_E9 = np.clip(I_restored_E9, 0, 1)

    # Restore Image (E11)
    I_restored_E11 = fineTunedE11_model.predict(I_blurry_array)
    I_restored_E11 = np.clip(I_restored_E11, 0, 1)

    # Normalize restored image
    I_restored_scaled_E9 = (I_restored_E9[0] * 255).astype('uint8')
    I_restored_scaled_E11 = (I_restored_E11[0] * 255).astype('uint8')

    # Create new extension for restored image
    new_filename = filename[:-4] + '_FTE9_restored.png'
    new_filename_E11 = filename[:-4] + '_FTE11_restored.png'

    save_img(
        os.path.join(RESTORED_PATH, new_filename),
        I_restored_scaled_E9,
        scale = False
    )

    save_img(
        os.path.join(RESTORED_PATH, new_filename_E11),
        I_restored_scaled_E11,
        scale = False
    )